In [1]:
from functools import wraps
from typing import Any, Type
from qcodes import Instrument, Parameter
from qcodes.instrument.base import InstrumentBase
from qcodes.instrument.parameter import _BaseParameter

In [2]:
class AbstractParameter(Parameter):
    """
    This is a trivial subclass of 'Parameter' to signal
    that this parameters *must* be overridden in
    subclasses of abstract instruments.
    """


class AbstractParameterException(Exception):
    """
    Errors with abstract parameter utilization should
    raise this exception class.
    """

In [3]:
class AbstractInstrument(Instrument): 
    
    def __new__(cls, *args, **kwargs): 
        instance = super().__new__(cls)
        cls.__init__(instance, *args, **kwargs)
        cls.__check_abstracts__(instance)
        return instance
    
    def __check_abstracts__(self): 
        
        abstract_parameters = [
            parameter.name for parameter in self.parameters.values()
            if isinstance(parameter, AbstractParameter)
        ]

        if any(abstract_parameters):
            cls_name = type(self).__name__

            raise AbstractParameterException(
                f"Class '{cls_name}' has un-implemented Abstract Parameter(s): " +
                ", ".join([f"'{name}'" for name in abstract_parameters])
            )
    
    def add_parameter(
            self, name: str, parameter_class: Type[_BaseParameter] = Parameter,
            **kwargs: Any
    ) -> None:

        existing_parameter = self.parameters.get(name, None)

        if isinstance(existing_parameter, AbstractParameter):
            # For abstract parameters, we define special behavior.
            existing_unit = getattr(existing_parameter, "unit", None)
            new_unit = kwargs.get("unit", None)

            if existing_unit and existing_unit != new_unit:
                raise AbstractParameterException(
                    f"The unit of the parameter '{name}' is '{new_unit}', "
                    f"which is inconsistent with the unit '{existing_unit}' "
                    f"specified in the baseclass {cls.__name__!r} "
                )

            # Remove the original abstract parameter to make room for the implementation
            # in the subclass.
            del self.parameters[name]

        super().add_parameter(
            name, parameter_class=parameter_class, **kwargs
        )

In [4]:
class AbstractVoltage(AbstractInstrument): 
    
    def __init__(self, name): 
        super().__init__(name)
    
        self.add_parameter(
            "voltage",
            parameter_class=AbstractParameter,
            unit="V"
        )
    

In [5]:
AbstractVoltage("name2")

AbstractParameterException: Class 'AbstractVoltage' has un-implemented Abstract Parameter(s): 'voltage'

In [6]:
class VoltageSource(AbstractVoltage): 
    def __init__(self, name):
        super().__init__(name)
        
        self.add_parameter(
            "voltage", 
            unit="V", 
            set_cmd=None, 
            get_cmd=None
        )

In [7]:
instrument = VoltageSource("name400")

KeyError: 'Another instrument has the name: name400'

In [8]:
VoltageSource._all_instruments

{'name2': <weakref at 0x000002107FB5AAE0; to 'AbstractVoltage' at 0x000002106E673130>,
 'name400': <weakref at 0x000002107FC744F0; to 'VoltageSource' at 0x000002107FB6E100>}

In [8]:
instrument.voltage(0.1)

AttributeError: 'NoneType' object has no attribute 'voltage'